# Analysis

After generating the models for all the algorithmns for different runtimes, csv files are generated for each models with the hyperparameters values. These hyperparameters are analysed to find three things:

1. Trying to find the important hyperparameters
2. Trying to find the range of hyperparameters
3. Compare the range of values across the models for different hyperparameters

In [62]:
#importing
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

import warnings
warnings.filterwarnings('ignore')



In [63]:
# importing libraries
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import re

In [64]:
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=5,port=port_no)

Checking whether there is an H2O instance running at http://localhost:51194 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/sv/0gx23csn4gg8nl275z72ss3w0000gn/T/tmpqi18wkit
  JVM stdout: /var/folders/sv/0gx23csn4gg8nl275z72ss3w0000gn/T/tmpqi18wkit/h2o_nupur_started_from_python.out
  JVM stderr: /var/folders/sv/0gx23csn4gg8nl275z72ss3w0000gn/T/tmpqi18wkit/h2o_nupur_started_from_python.err
  Server is running at http://127.0.0.1:51194
Connecting to H2O server at http://127.0.0.1:51194 ... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,25 days
H2O cluster name:,H2O_from_python_nupur_kg7jdr
H2O cluster total nodes:,1
H2O cluster free memory:,4.792 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


Data set is imported using pandas as it is needed for GridSearch to find important hyperparameters

In [65]:
dfd=pd.read_csv("anime_filtered.csv", decimal = ',')

In [66]:
dfd.dtypes

anime_id           int64
title             object
title_english     object
title_japanese    object
title_synonyms    object
image_url         object
type              object
source            object
episodes           int64
status            object
airing              bool
aired_string      object
aired             object
duration          object
rating            object
score             object
scored_by          int64
rank              object
popularity         int64
members            int64
favorites          int64
background        object
premiered         object
broadcast         object
related           object
producer          object
licensor          object
studio            object
genre             object
opening_theme     object
ending_theme      object
dtype: object

As the datatypes of some real values are object, we have to convert all of them to float

In [67]:
dfd['score'] = dfd.score.astype(float)

In [68]:
ser = pd.Series(dfd['rank'])
dfd['rank'] = ser.astype('float')

Categorical columns and boolean values are converted to integers and 

In [69]:
dfd['status'] = dfd['status'].astype('category').cat.codes
dfd['source'] = dfd['source'].astype('category').cat.codes
dfd['type'] = dfd['type'].astype('category').cat.codes

In [70]:
dfd = dfd.applymap(lambda x: 1 if x == True else x)
dfd = dfd.applymap(lambda x: 0 if x == False else x)

In [71]:
dfd.dtypes

anime_id            int64
title              object
title_english      object
title_japanese     object
title_synonyms     object
image_url          object
type                int64
source              int64
episodes            int64
status              int64
airing              int64
aired_string       object
aired              object
duration           object
rating             object
score             float64
scored_by           int64
rank              float64
popularity          int64
members             int64
favorites           int64
background         object
premiered          object
broadcast          object
related            object
producer           object
licensor           object
studio             object
genre              object
opening_theme      object
ending_theme       object
dtype: object

X and Y are initialised with dataset columns with score as target

In [72]:
columns = ['type','source','episodes','status','airing','scored_by','popularity','members','favorites']
X = dfd[columns]
y = dfd.score

__XGBoost__ 

In [ ]:
#importing the file
# df=pd.read_csv("XGboost.csv", decimal = ',')
df = h2o.import_file(path="XGboost.csv")

In [ ]:
df.head()

To pass the parameters in grid search, we need all the values in a list. Hence all the unique values of ntrees,minrows,col_sam_r_per_tree are taken and converted to list.

In [ ]:
pd = df.as_data_frame()
ntrees = pd['ntrees'].unique().tolist()

minrows = pd['min_rows'].unique().tolist()
col_sam_r_per_tree = pd['col_sam_r_per_tree'].unique().tolist()

In [ ]:
df.types

Here, we are finding the range of all the column which are numerical and added to a list called ranges.

In [ ]:
ranges = []
for key,val in df.types.items():
    print(key,val)
    if val == 'int': 
         ranges.append([key,[df[key].min(),df[key].max()]])
    if val == 'real':
         ranges.append([key,[df[key].min(),df[key].max()]])

__2 Trying to find the range of hyperparameters__

In [ ]:
ranges

__1 Trying to find the important hyperparameters__

In Grid Search CV , XGBRegressor is taken and the parameters which can be passed in GridSearch are initialised and the model is fitted with X and y.

In [18]:

from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
# Create the parameter grid based on the results of random search 
param_grid = {
    'ntrees': ntrees,
    'stopping_rounds': [3],
    'max_depth' : [10],
    'learn_rate': [0.05],
    'min_rows': 10.0,
    'sample_rate': [0.8],
    'col_sample_rate': [0.8],
    'col_sample_rate_per_tree' : [0.8]

}
# Create a based model
clf = XGBRegressor()
# Instantiate the grid search model
xgb = GridSearchCV(estimator = clf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
re = xgb.fit(X,y)

Fitting 3 folds for each of 24192 candidates, totalling 72576 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.0min


These are the best parameters for XGB given by GridSearch 

In [ ]:
xgb.best_params_

In [ ]:
means = res.cv_results_['mean_test_score']
stds = res.cv_results_['std_test_score']
params = res.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

__GBM__

GBM hyperparameters are imported for the analysis 

In [166]:
#importing the file
df = h2o.import_file(path="GBM.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [167]:
df.head(40)

gbm_id,name,nfolds,max_depth,min_rows,distribution,stopping_metric,stopping_tolerance,seed,sample_rate
1,GBM_grid_1_AutoML_20190418_142243_model_3,5,6,6,gaussian,deviance,0.008312,-7.92e+18,0.6
2,GBM_1_AutoML_20190418_144229,5,15,15,gaussian,deviance,0.008312,3.08e+18,0.8
3,GBM_grid_1_AutoML_20190418_142243_model_2,5,10,10,gaussian,deviance,0.008312,3.07e+18,0.7
4,GBM_grid_1_AutoML_20190418_140346_model_3,5,4,4,gaussian,deviance,0.008312,-5.29e+18,0.5
5,GBM_3_AutoML_20190418_140346,5,15,15,gaussian,deviance,0.008312,-7.17e+18,0.8
6,GBM_5_AutoML_20190418_150256,5,15,15,gaussian,deviance,0.008312,-5.05e+18,0.8
7,GBM_grid_1_AutoML_20190418_144229_model_4,5,12,12,gaussian,deviance,0.008312,3.42e+18,0.6
8,GBM_2_AutoML_20190418_140346,5,15,15,gaussian,deviance,0.008312,-7.17e+18,0.8
9,GBM_2_AutoML_20190418_144229,5,15,15,gaussian,deviance,0.008312,3.08e+18,0.8
10,GBM_4_AutoML_20190418_142243,5,15,15,gaussian,deviance,0.008312,3.67e+18,0.8


Some columns are converted to list for Grid Search 

In [168]:
pd = df.as_data_frame()
minrows = pd['min_rows'].unique().tolist()
maxdepth = pd['max_depth'].unique().tolist()
sample_rate = pd['sample_rate'].unique().tolist()

In [170]:
maxdepth

[6, 15, 10, 4, 12, 7, 3, 8, 5, 16, 13, 11]

__2 Trying to find the range of hyperparameters__

In [171]:
df.types

{'gbm_id': 'int',
 'name': 'string',
 'nfolds': 'int',
 'max_depth': 'int',
 'min_rows': 'int',
 'distribution': 'enum',
 'stopping_metric': 'enum',
 'stopping_tolerance': 'real',
 'seed': 'int',
 'sample_rate': 'real'}

Here, we are finding the range of all the column which are numerical and added to a list called ranges.

In [172]:
ranges = []
for key,val in df.types.items():
    print(key,val)
    if val == 'int': 
         ranges.append([key,[df[key].min(),df[key].max()]])
    if val == 'real':
         ranges.append([key,[df[key].min(),df[key].max()]])

gbm_id int
name string
nfolds int
max_depth int
min_rows int
distribution enum
stopping_metric enum
stopping_tolerance real
seed int
sample_rate real


Here are the ranges for GBM hyperparameters.

In [173]:
ranges

[['gbm_id', [1.0, 39.0]],
 ['nfolds', [5.0, 5.0]],
 ['max_depth', [3.0, 16.0]],
 ['min_rows', [3.0, 16.0]],
 ['stopping_tolerance', [0.008312003, 0.008312003]],
 ['seed', [-9e+18, 8.93e+18]],
 ['sample_rate', [0.5, 1.0]]]

__1. Trying to find the important hyperparameters__

Only max_depth is passed as it was not working for other parameters.

In [174]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth' : [6, 15, 10, 4, 12, 7, 3, 8, 5, 16, 13, 11],
     'nfolds' :  [5.0],
    'max_depth': [ 10.0],
    'min_rows': [12.0],
    'stopping_tolerance' : [0.008312003],
     'seed': [90000],
     'sample_rate': [0.8]

}
# Create a based model
rgs = GradientBoostingRegressor()
# Instantiate the grid search model
gbm = GridSearchCV(estimator = rgs, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [176]:
res = gbm.fit(x,y)

ValueError: Found input variables with inconsistent numbers of samples: [30, 5]

The best hyperparameter value for max_depth is found to be 5 

In [184]:
gbm.best_params_

{'max_depth': 5}

In [185]:
means = res.cv_results_['mean_test_score']
stds = res.cv_results_['']
params = res.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
    
    
    

0.774989 (0.047958) with: {'max_depth': 6}
0.725470 (0.069039) with: {'max_depth': 15}
0.768193 (0.049578) with: {'max_depth': 10}
0.777660 (0.030956) with: {'max_depth': 4}
0.755503 (0.052856) with: {'max_depth': 12}
0.776038 (0.047122) with: {'max_depth': 7}
0.771216 (0.027889) with: {'max_depth': 3}
0.777515 (0.042712) with: {'max_depth': 8}
0.784473 (0.027866) with: {'max_depth': 5}
0.721390 (0.066199) with: {'max_depth': 16}
0.746037 (0.059102) with: {'max_depth': 13}
0.762322 (0.051540) with: {'max_depth': 11}


__GLM__

In [151]:
df_anime = h2o.import_file(path="anime_filtered.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [152]:
X =df_anime
def check_X(x,df):
    for name in x:
        if name not in df.columns:
            x.remove(name)  
    return x    

X=check_X(X,df_anime)

GLM hyperparameters file is imported for analysis.

In [153]:
#importing the file
df = h2o.import_file(path="GLM.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [154]:
df.head()

glm_id,name,nfolds,seed,solver,kc_val_mo,kc_val_pr,fold_assignment,lambda_search,max_iterations,objective_epsilon,gradient_epsilon,link,lambda_min_ratio,obj_reg
1,GLM_grid_1_AutoML_20190418_132524_model_1,5,3.52e+18,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05
2,GLM_grid_1_AutoML_20190418_135050_model_1,5,-7.58e+16,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05
3,GLM_grid_1_AutoML_20190418_140346_model_1,5,6.09e+18,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05
4,GLM_grid_1_AutoML_20190418_142243_model_1,5,4.02e+18,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05
5,GLM_grid_1_AutoML_20190418_144229_model_1,5,6.33e+18,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05
6,GLM_grid_1_AutoML_20190418_150256_model_1,5,-5.91e+18,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05


__2 Trying to find the range of hyperparameters__

In [155]:
df.types

{'glm_id': 'int',
 'name': 'string',
 'nfolds': 'int',
 'seed': 'int',
 'solver': 'enum',
 'kc_val_mo': 'enum',
 'kc_val_pr': 'enum',
 'fold_assignment': 'enum',
 'lambda_search': 'enum',
 'max_iterations': 'int',
 'objective_epsilon': 'int',
 'gradient_epsilon': 'real',
 'link': 'enum',
 'lambda_min_ratio': 'real',
 'obj_reg': 'real'}

Here, we are finding the range of all the column which are numerical and added to a list called ranges.

In [156]:
ranges = []
for key,val in df.types.items():
    print(key,val)
    if val == 'int': 
         ranges.append([key,[df[key].min(),df[key].max()]])
    if val == 'real':
         ranges.append([key,[df[key].min(),df[key].max()]])

glm_id int
name string
nfolds int
seed int
solver enum
kc_val_mo enum
kc_val_pr enum
fold_assignment enum
lambda_search enum
max_iterations int
objective_epsilon int
gradient_epsilon real
link enum
lambda_min_ratio real
obj_reg real


In [157]:
ranges

[['glm_id', [1.0, 6.0]],
 ['nfolds', [5.0, 5.0]],
 ['seed', [-5.91e+18, 6.33e+18]],
 ['max_iterations', [300.0, 300.0]],
 ['objective_epsilon', [300.0, 300.0]],
 ['gradient_epsilon', [1e-06, 1e-06]],
 ['lambda_min_ratio', [0.0001, 0.0001]],
 ['obj_reg', [6.91e-05, 6.91e-05]]]

In [116]:
param_grid = {
    'stopping_rounds': [3],
    'seed' : [-500000, -1000, 10, 800, 1000, 20000, 600000]
}

In [117]:
df_anime = h2o.import_file(path='anime_filtered.csv')
x = [name for name in df_anime.columns if name != 'score']
y='score'


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [118]:
from h2o.estimators import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch
# Train and validate a cartesian grid of GBMs
drf_grid1 = H2OGridSearch(model=H2OGeneralizedLinearEstimator,
                         hyper_params=param_grid)

In [119]:
drf_grid1.train(x,y, training_frame=df_anime)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [121]:
drf_grid1 = drf_grid1.get_grid(sort_by='rmse')
drf_grid1

        seed stopping_rounds  \
0         10               3   
1      20000               3   
2     600000               3   
3    -500000               3   
4       1000               3   
5        800               3   
6      -1000               3   

                                                           model_ids  \
0  Grid_GLM_anime_filtered8.hex_model_python_1556327508338_6_model_3   
1  Grid_GLM_anime_filtered8.hex_model_python_1556327508338_6_model_6   
2  Grid_GLM_anime_filtered8.hex_model_python_1556327508338_6_model_7   
3  Grid_GLM_anime_filtered8.hex_model_python_1556327508338_6_model_1   
4  Grid_GLM_anime_filtered8.hex_model_python_1556327508338_6_model_5   
5  Grid_GLM_anime_filtered8.hex_model_python_1556327508338_6_model_4   
6  Grid_GLM_anime_filtered8.hex_model_python_1556327508338_6_model_2   

                 rmse  
0  1.0619433240997063  
1  1.0619433240997063  
2  1.0619433240997063  
3  1.0619433240997063  
4  1.0619433240997063  
5  1.0619433240997063 

__From the above scenario we can indicate that as the with the change of seed hyperparameter for the GLM model, there is no change in the RMSE value.RMSE value comes out to be constant. Therefore, we can say that, seed is not an important hyperparameter for the the GLM model!__

__XRT__

Importing XRT hyperparameter file for the analysis.

In [158]:
#importing the file
df = h2o.import_file(path="XRT.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [159]:
df.head()

xrt_id,name,nfolds,seed,kc_val_mo,kc_val_pr,fold_assignment,ntrees,distributionstopping_metric,stopping_tolerance,histogram_type
1,XRT_1_AutoML_20190418_132524,5,-6.68e+18,FALSE,TRUE,Modulo,32,nan,0.008312,Random
2,XRT_1_AutoML_20190418_135050,5,6.82e+18,FALSE,TRUE,Modulo,46,nan,0.008312,Random
3,XRT_1_AutoML_20190418_140346,5,6.72e+18,FALSE,TRUE,Modulo,14,nan,0.008312,Random
4,XRT_1_AutoML_20190418_142243,5,2.63e+18,FALSE,TRUE,Modulo,18,nan,0.008312,Random
5,XRT_1_AutoML_20190418_144229,5,2.04e+18,FALSE,TRUE,Modulo,50,nan,0.008312,Random
6,XRT_1_AutoML_20190418_150256,5,-7.04e+18,FALSE,TRUE,Modulo,50,nan,0.008312,Random


Stored the values of ntrees values in list for Grid Search

In [160]:
pd = df.as_data_frame()
ntrees = pd['ntrees'].unique().tolist()


__2 Trying to find the range of hyperparameters__

In [161]:
df.types

{'xrt_id': 'int',
 'name': 'string',
 'nfolds': 'int',
 'seed': 'int',
 'kc_val_mo': 'enum',
 'kc_val_pr': 'enum',
 'fold_assignment': 'enum',
 'ntrees': 'int',
 'distributionstopping_metric': 'int',
 'stopping_tolerance': 'real',
 'histogram_type': 'enum'}

Here, we are finding the range of all the column which are numerical and added to a list called ranges.

In [162]:
ranges = []
for key,val in df.types.items():
    print(key,val)
    if val == 'int': 
         ranges.append([key,[df[key].min(),df[key].max()]])
    if val == 'real':
         ranges.append([key,[df[key].min(),df[key].max()]])

xrt_id int
name string
nfolds int
seed int
kc_val_mo enum
kc_val_pr enum
fold_assignment enum
ntrees int
distributionstopping_metric int
stopping_tolerance real
histogram_type enum


In [163]:
ranges

[['xrt_id', [1.0, 6.0]],
 ['nfolds', [5.0, 5.0]],
 ['seed', [-7.04e+18, 6.82e+18]],
 ['ntrees', [14.0, 50.0]],
 ['distributionstopping_metric', [nan, nan]],
 ['stopping_tolerance', [0.008312003, 0.008312003]]]

Tried to do Grid Search to find important hyperparameter in ntrees for XRT model but it did not work and as per H2O documentation, they have not defined hypeparameters for the XRT model

In [164]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'ntrees' : [32, 46, 14, 18, 50]

}
# Create a based model
etr = ExtraTreesRegressor()
# Instantiate the grid search model
reg = GridSearchCV(estimator = etr, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [165]:
reg.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

__DRF__

In [139]:
#importing the file
df = h2o.import_file(path="DRF.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [140]:
df.head()

drf_id,name,nfolds,keep_cross_validation_models,keep_cross_validation_predictions,fold_assignment,stopping_metric,stopping_tolerance,seed,distribution
1,DRF_1_AutoML_20190418_132524,5,FALSE,TRUE,Modulo,AUTO,0.008312,-6.25e+18,gaussian
2,DRF_1_AutoML_20190418_135050,5,FALSE,TRUE,Modulo,deviance,0.008312,-4.44e+18,gaussian
3,DRF_1_AutoML_20190418_140346,5,FALSE,TRUE,Modulo,deviance,0.008312,2.36e+18,gaussian
4,DRF_1_AutoML_20190418_142243,5,FALSE,TRUE,Modulo,deviance,0.008312,-1.13e+18,gaussian
5,DRF_1_AutoML_20190418_144229,5,FALSE,TRUE,Modulo,deviance,0.008312,3.86e+18,gaussian
6,DRF_1_AutoML_20190418_150256,5,FALSE,TRUE,Modulo,deviance,0.008312,8.56e+18,gaussian


__2 Trying to find the range of hyperparameters__

In [141]:
df.types

{'drf_id': 'int',
 'name': 'string',
 'nfolds': 'int',
 'keep_cross_validation_models': 'enum',
 'keep_cross_validation_predictions': 'enum',
 'fold_assignment': 'enum',
 'stopping_metric': 'enum',
 'stopping_tolerance': 'real',
 'seed': 'int',
 'distribution': 'enum'}

Here, we are finding the range of all the column which are numerical and added to a list called ranges.

In [142]:
ranges = []
for key,val in df.types.items():
    print(key,val)
    if val == 'int': 
         ranges.append([key,[df[key].min(),df[key].max()]])
    if val == 'real':
         ranges.append([key,[df[key].min(),df[key].max()]])

drf_id int
name string
nfolds int
keep_cross_validation_models enum
keep_cross_validation_predictions enum
fold_assignment enum
stopping_metric enum
stopping_tolerance real
seed int
distribution enum


In [143]:
ranges

[['drf_id', [1.0, 6.0]],
 ['nfolds', [5.0, 5.0]],
 ['stopping_tolerance', [0.008312003, 0.008312003]],
 ['seed', [-6.25e+18, 8.56e+18]]]

__Important hyperparameters for grid search for the model DRF are mtries and categorical_encoding, these are the hyperparameters that are not a part of our model__

__3 Compare the range of values across the models for different hyperparameters__ :

1 nfolds 

nfolds value is same for all the algorithms  

2 seed 

seed value ranged from -9e+18 to 8.93e+18 across the models for all the algorithmns

3 stopping_tolerance

It is present is all the algorithmns except GLM and its value is 0.008312003 for all models

4 ntrees

ntress are the hyperparameters for only XGBoost and XRT and the range for XGBoost is from 13 to 276 while for XRT is form 15 to 50 

XGboost and GBM has some same hyperparameters such as max_depth, min_rows, sample_rate

- max_depth ranges for both algorithms are almost similar XGBoost - (5,20) & GBM - (3,16)

- min_rows - XGboost values varied from 0.01 to 20 while for GBM its from 3 to 16  

- sample_rate - both algorithmns are same sample_rate of 0.5 to 1 



